Importamos todos los paquetes que necesitamos

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

In [ ]:
hbar = 1.0

In [ ]:
nsites = 1002
temp = 0.01
mu = 0.0
beta = -1.0
delta = -0.25
gamma = -0.10

In [ ]:
nsteps = 2100
dt = 0.1
DeltaV = 0.5

In [ ]:
def kdelta(k, l):
    if k == l: return True
    else: return False

In [ ]:
def fermif(energy, mu, temp):
    expo = (energy-mu)/temp
    if expo < -100.0:
        return 1.0
    elif expo > 100.0:
        return 0.0
    else:
        return 1.0/(math.exp(expo)+1.0)

In [ ]:
DeltaVs = np.linspace(0.0,1.0,20)

currents = []

for ii, DeltaV in enumerate(DeltaVs):
    
    # Print something to let us know it's alive
    
    print('Starting with ',ii,' DeltaV = ',DeltaV)
    
    # Build Biased H
    
    h = np.zeros((nsites,nsites))

    for i in range(0,nsites):
        for j in range (0,nsites):
            if kdelta(i,j-1):
                h[i,j] = beta
            elif kdelta(i,j+1):
                h[i,j] = beta
            elif kdelta(i,j) and i < int(nsites/2) - 1:
                h[i,j] = DeltaV / 2
            elif kdelta(i,j) and i > int(nsites/2) + 0:
                h[i,j] = - DeltaV / 2
            
    h[int(nsites/2)-2,int(nsites/2)-1] = gamma
    h[int(nsites/2)-1,int(nsites/2)  ] = delta
    h[int(nsites/2)  ,int(nsites/2)+1] = gamma

    h[int(nsites/2)-1,int(nsites/2)-2] = gamma
    h[int(nsites/2)  ,int(nsites/2)-1] = delta
    h[int(nsites/2)+1,int(nsites/2)  ] = gamma
    
    # Build biased GS DM
    
    w, v = np.linalg.eig(h.real)
    rho_diag = np.zeros((nsites, nsites))
    for i in range(0, nsites):
        rho_diag[i, i] = fermif(w[i], mu, temp)
    rho_0 = np.dot(v, np.dot(rho_diag, np.transpose(v)))
    
    # placeholders for output
    
    time = []
    current = []
    
    # Boot integrator
    
    rho = rho_0
    rhodot = ((0-1.0j)/hbar)*(np.dot(h,rho)-np.dot(rho,h))
    rhoold = rho - dt*rhodot
    
    # Build propagation H
    
    np.fill_diagonal(h,0.0)

    h[int(nsites/2)-2,int(nsites/2)-1] = gamma
    h[int(nsites/2)-1,int(nsites/2)  ] = delta
    h[int(nsites/2)  ,int(nsites/2)+1] = gamma

    h[int(nsites/2)-1,int(nsites/2)-2] = gamma
    h[int(nsites/2)  ,int(nsites/2)-1] = delta
    h[int(nsites/2)+1,int(nsites/2)  ] = gamma
    
    # Do time propagation for nsteps
    
    for step in range(0,nsteps):
        if (step % 1000) == 0:
            print('Integration step ',step,' of ',nsteps,' in DeltaV ',DeltaV)
        rhodot = ((0-1.0j)/hbar)*(np.dot(h,rho)-np.dot(rho,h))
        rhonew = rhoold + 2.0 * dt * rhodot
        rhoold = rho
        rho = rhonew
        # calculate observables
        time.append(step*dt)
        current.append(4.0 * h[int(nsites/2)-1,int(nsites/2)].real * rho[int(nsites/2),int(nsites/2)-1].imag)
        
    # Save current trace for this Delta V
    
    currents.append(np.array(current))
    
    # Print something to let us know it's alive
    
    print('Done with ',i,' DeltaV = ',DeltaV)

In [ ]:
for i,DeltaV in enumerate(DeltaVs):
    plt.plot(time,currents[i])
plt.show()

In [ ]:
ivsv = []
for i,DeltaV in enumerate(DeltaVs):
    ivsv.append(np.average(currents[i][-1000:-1]))

In [ ]:
dv_gf = np.loadtxt('deltavs.dat')
ivsv_gf = np.loadtxt('ivsv.dat')

In [ ]:
plt.plot(DeltaVs,ivsv)
plt.plot(dv_gf,ivsv_gf)
plt.show()

In [ ]:
dv_gf_ext = np.loadtxt('deltavs-ext.dat')
ivsv_gf_ext = np.loadtxt('ivsv-ext.dat')

In [ ]:
plt.plot(DeltaVs,ivsv)
plt.plot(dv_gf_ext,ivsv_gf_ext)
plt.show()